In [25]:
import h5py
import chabud
from chabud import BandExtractor, NBR, NDVI, GNDVI, EVI, AVI, SAVI, NDMI, MSI, GCI, BSI, NDWI, NDSI, NDGI
import numpy as np
from numpy.typing import NDArray
import xarray as xr
import pytorch_lightning as pl

import pandas as pd
from trimesh.voxel.runlength import dense_to_brle
from collections import defaultdict

from typing import Any, Union, Dict, Literal
from typing import List, Tuple
from numpy.typing import NDArray

import torch

from pathlib import Path
from tqdm.auto import tqdm

In [2]:
BASEDIR = Path("/global/public/chabud-ecml-pkdd2023/")
fn = BASEDIR / "train_eval.hdf5"

In [3]:
#show checkpoints
#CHECK_BASEDIR = Path("./lightning_logs/version_9")

#CHECKPOINTDIR = CHECK_BASEDIR / "checkpoints"

#list(CHECKPOINTDIR.glob("*.ckpt"))

In [4]:
model = chabud.FireModel.load_from_checkpoint(Path('lightning_logs/version_9/checkpoints/epoch=39-step=1040.ckpt'), map_location="cpu")

In [5]:
def compute_submission_mask(id: str, mask: NDArray):
    brle = dense_to_brle(mask.astype(bool).flatten())
    return {"id": id, "rle_mask": brle, "index": np.arange(len(brle))}

def obtain_submission_df(submissions: Dict[str, NDArray]) -> pd.DataFrame:
    res = []
    for uuid, prediction in submissions.items():
      submission_mask = compute_submission_mask(uuid, prediction)
      res.append(pd.DataFrame(submission_mask))
    return pd.concat(res)

In [6]:
def to_xarray(dataset, pretty_band_names=True):
    """Convert a single example into an xarray for easy access"""
    
    if pretty_band_names:
        BANDS = ["coastal_aerosol", "blue", "green", "red",
                 "veg_red_1", "veg_red_2", "veg_red_3", "nir", 
                 "veg_red_4", "water_vapour", "swir_1", "swir_2"]
    else:
        BANDS = ["1", "2", "3", "4", "5", "6", "7", "8", "8a", "9", "11", "12"]
        
    post = dataset["post_fire"][...].astype("float32") / 10000.0
    
    # Da `pre_fire` manchmal fehlt ersetzen wir es durch 0 Werte was
    # eh der Platzhalter für einen fehlenden Messwert ist.
    try:
        pre = dataset["pre_fire"][...].astype("float32") / 10000.0
    except KeyError:
        pre = np.zeros_like(post, dtype="float32")
        
    # Da die Maske nur ein "Band" hat können wir die dritte Dimension einfach
    # weglassen. Das erreichen wir in dem wir mit `0` am Ende indizieren.
    mask = dataset["mask"][..., 0].astype("bool")
    
    return {"pre": xr.DataArray(pre, dims=["x", "y", "band"], coords={"x": range(512), "y": range(512), "band": BANDS}),
            "post": xr.DataArray(post, dims=["x", "y", "band"], coords={"x": range(512), "y": range(512), "band": BANDS}),
            "mask": xr.DataArray(mask, dims=["x", "y"], coords={"x": range(512), "y": range(512)}),
            "fold": dataset.attrs["fold"]}

In [7]:
class PPModel:
    def _init_(self, model):
        self._model = model
        self._model.eval()
        
    def _call_(self, bands) -> Any:
        # preprocessing
        bands = bands / 10000
        channels = np.stack([c(bands) for c in self._model.channels])
        channels = torch.Tensor(channels())
        # Modell auswerten
        with torch.set_grad_enabled(False):
            mask = self.model.forward(channels.sigmoid() > 0.5)
        # Postprocessing
        mask = mask[0, 0, ...].detach.numpy()
        return mask

In [ ]:
def process_dataset(bands, true):
    channels = np.stack([c(bands) for c in mdl.channels])
    with torch.set_grad_enabled(False):
        pred = mdl.forward(torch.Tensor(channels[np.newaxis, ...])).sigmoid() > 0.5
        pred = pred[0, 0, ...].detach().numpy()


In [30]:
def loader():
    post = []

    # Read hdf5 file and filter by fold
    with h5py.File(fn, "r") as fd:
        for uuid, values in fd.items():
            if values.attrs['fold'] != 0:
                continue                  
            if "pre_fire" not in values:
                continue

            post.append(values["post_fire"][...])

    # Convert to numpy arrays
    post = np.stack(post, axis=0, dtype=np.int32)
    return post

def compute_submission_mask(id: str, mask: NDArray):
    brle = dense_to_brle(mask.astype(bool).flatten())
    return {"id": id, "rle_mask": brle, "index": np.arange(len(brle))}

if __name__ == '__main__':
    validation_fold = loader()
    # use a list to accumulate results
    result = []
    # instantiate the model
    model = chabud.FireModel.load_from_checkpoint(Path('lightning_logs/version_9/checkpoints/epoch=39-step=1040.ckpt'), map_location="cpu")
    #for uuid,item in validation_fold:
        # model.forward()
        # perform the prediction
        # print(input_images['post'].shape)
        
    channels = np.stack([c(loader()) for c in model.channels])
    with torch.set_grad_enabled(False):
        pred = model.forward(torch.Tensor(channels[np.newaxis, ...])).sigmoid() > 0.5
        pred = pred[0, 0, ...].detach().numpy()
        # convert the prediction in RLE format
        encoded_prediction = compute_submission_mask(uuid, pred)
        result.append(pd.DataFrame(encoded_prediction))

    # concatenate all dataframes
    submission_df = pd.concat(result)
    submission_df.to_csv('predictions.csv', index=False)

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [1, 5, 78, 512, 512]

In [ ]:
#class PPModel:
#    def _init_(self, model):
#        self._model = model
#        self._model.eval()
        
#    def _call_(self, inut) -> Any:
        #Preprocessing
#        bands = bands / 10000
#        channels = np.stack([])
#        mask_logits = self._model.forward(input)
        # Mein PP
        
#        return mask